In [0]:
from IPython.core.interactiveshell import InteractiveShell

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

## Setup

In [2]:
!curl -s https://course.fast.ai/setup/colab | bash
!git clone https://github.com/SpellOnYou/dlff-note.git
import os
os.chdir('/content/dlff-note/nbs/dl2/')

Updating fastai...
Done.
Cloning into 'dlff-note'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 5313 (delta 11), reused 24 (delta 10), pack-reused 5282
Receiving objects: 100% (5313/5313), 240.19 MiB | 33.83 MiB/s, done.
Resolving deltas: 100% (2872/2872), done.


In [0]:
%%script false
pip freeze

In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)


In [0]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [6]:
x_train,y_train,x_valid,y_valid = get_data()

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [0]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [9]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [13]:
x_valid.mean(), x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [0]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [10]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [0]:
nh = 50
def lin(x, w, b): return x@w + b;
w1 = torch.randn(m,nh)*math.sqrt(2./m ); b1 = torch.zeros(nh)
w2 = torch.randn(nh,1); b2 = torch.zeros(1)

def relu(x): return x.clamp_min(0.) - 0.5
t1 = relu(lin(x_valid, w1, b1))

def model(xb):
    l1 = lin(xb, w1, b1) # one linear layer
    l2 = relu(l1) # one relu layer
    l3 = lin(l2, w2, b2) # one more linear layer
    return l3

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
# num hidden
nh = 50

[Thinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

##### Just grap some normal random numbers with linear function

In [0]:
def lin(x, w, b): return x@w + b

In [0]:
# If we do not divide by sqrt
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
t = lin(x_valid, w1, b1) # hidden
t.mean(), t.std()

(tensor(-1.3050), tensor(26.8068))

In [0]:
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)
t2 = lin(t, w2, b2) # output
t2.mean(), t2.std()

(tensor(-72.2154), tensor(128.5003))

In [0]:
t.shape, t2.shape

(torch.Size([10000, 50]), torch.Size([10000, 1]))

##### Kaiming Initialization

In [0]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [0]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [0]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
t = lin(x_valid, w1, b1)

In [0]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.1049), tensor(0.9489))

In [0]:
def relu(x): return x.clamp_min(0.) 

In [0]:
t = relu(lin(x_valid, w1, b1))

In [0]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.4302), tensor(0.5891))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)
w1.mean(),w1.std()

t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5733), tensor(0.8442))

##### Torch package

In [0]:
#export
from torch.nn import init

In [0]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') # why fan_out??
t = relu(lin(x_valid, w1, b1))

In [0]:
# init.kaiming_normal_??
help(init.kaiming_normal_)

Help on function kaiming_normal_ in module torch.nn.init:

kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
    Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where
    
    .. math::
        \text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan\_in}}}
    
    Also known as He initialization.
    
    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``'fan_out'`` preserves the magnitudes in t

See the 
- choosing 'fan_in' preserves the magnitude of the variance of the wights in the forward pass
- choosing 'fan_out' preserves the magnitues in the backward pass

In [0]:
w1.shape

torch.Size([784, 50])

In [0]:
import torch.nn

In [0]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

**We should do fan_out(backward); cz pytorch use with transposed matrix**

In [0]:
# torch.nn.Linear.forward?? #what is this? why jeremy input this?
# help(torch.nn.Linear.forward)

In [0]:
%%script false
torch.nn.functional.linear??

In [0]:
%%script false
help(torch.nn.functional.linear)

In [0]:
t.mean(),t.std() #adequate to linear layer's variate, which is Relu.

(tensor(0.5419), tensor(0.7970))

In [0]:
torch.nn.Conv2d??

In [0]:
%%script false
torch.nn.Conv2d.conv2d_forward?? #jiwon put here

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
def relu(x): return x.clamp_min(0.)

In [0]:
# what if...? mitigate mean bias...

def relu(x): return x.clamp_min(0.) - 0.5

In [0]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(-0.0717), tensor(0.7405))

Does it really enhance bias when we extract 0.5?? (just wanted to test jeremy)

WE can forward pass now!!!

1.   ReLU
2.   Linear
3.   Init





a model just can be a function

In [17]:
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()
def model(xb):
    l1 = lin(xb, w1, b1) # one linear layer
    l2 = relu(l1) # one relu layer
    l3 = lin(l2, w2, b2) # one more linear layer
    return l3

NameError: ignored

In [0]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 22.2 ms per loop


In [0]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1]) #(10000, 1)

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [19]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [0]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
y_train,y_valid = y_train.float(),y_valid.float()

In [0]:
preds = model(x_train)

In [0]:
preds.shape

torch.Size([50000, 1])

In [0]:
mse(preds, y_train)

tensor(32.0189)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

##### Jiwon Practice 😃

- forward

$x_1$ = torch.rand(n,m) = *x_train*;

$w_1$ = torch.rand(m,nh); $b_1$ = torch.rand(n,1)

$lin_1\ =\ x_1 @ w_1 + b_1$

$lin_2\ =\ ReLu(lin_1)) = x_2$

$w_2$ = torch.rand(nh,1); $b_2$ = torch.rand(1)

$lin_3 = \ x_2 @ w_2 + b_2$

In [0]:
lin1=lin(x_train, w1, b1)
lin2=relu(lin1) #this is x_2 (in case x_train is x_1)
lin3=lin(lin2,w2,b2) #lin2 @ w2 + b2
mse_loss = (lin3.squeeze(-1) - y_train).pow(2).mean()

In [24]:
print(f"{x_train.shape}, {w1.shape}, {b1.shape}\n")
print(f"{lin2.shape}, {w2.shape}, {b2.shape}\n")
f"{mse_loss.shape}, {lin3.shape}, {y_train.shape}"

torch.Size([50000, 784]), torch.Size([784, 50]), torch.Size([50])

torch.Size([50000, 50]), torch.Size([50, 1]), torch.Size([1])



'torch.Size([]), torch.Size([50000, 1]), torch.Size([50000])'

- backward - what happens in backwards process? (examplify)

$$\frac{\partial}{\partial\ u}MSE(u,y)\times\frac{\partial}{\partial v}l_2(v)\times\frac{\partial}{\partial t}ReLU(t)\times\frac{\partial}{\partial x}l_1(x)$$

##### MSE derivative

In [28]:
lin3.g=(lin3.squeeze(-1)-y_train).unsqueeze(-1);lin3.g.shape

torch.Size([50000, 1])

##### linear2 derivative

##### ReLU derivative

##### Linear1 derivative

##### linear3 derivative
lin3=lin(lin2,w2,b2) $$ lin2 @ w2 + b2$$

In [33]:
w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
b.g = out.g.sum(0)

NameError: ignored

In [0]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [0]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [0]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

##### Jiwon Practice 😀

In [0]:
#get the value from skeleton
tmp_l1 = lin(x_train, w1, b1)
tmp_l2 = relu(tmp_l1)
tmp_l3 = lin(tmp_l2, w2, b2)
# gradient backward(backprop)
mse_grad(inp=tmp_l3, targ=y_train) # tmp_l3.g, 
print(f"L3 grad: {tmp_l3.g.shape}")

lin_grad(inp=tmp_l2, out=tmp_l3, w=w2, b=b2) # use tmp_12
print(f"L2 grad: {tmp_l2.g.shape}, \n\tw2 grad: {w2.g.shape}, b2 grad: {b2.g.shape}")

relu_grad(inp= tmp_l1, out=tmp_l2)
print(f"L1 grad: {tmp_l1.g.shape}")

lin_grad(inp=x_train, out=tmp_l1, w=w1, b=b1)
print(f"x_train grad: {x_train.g.shape},\n\tw1 grad: {w1.g.shape}, b1 grad: {b1.g.shape}")

L3 grad: torch.Size([50000, 1])
L2 grad: torch.Size([50000, 50]), 
	w2 grad: torch.Size([50, 1]), b2 grad: torch.Size([1])
L1 grad: torch.Size([50000, 50])
x_train grad: torch.Size([50000, 784]),
	w1 grad: torch.Size([784, 50]), b1 grad: torch.Size([50])


In [0]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [0]:
forward_and_backward(x_train, y_train)

In [0]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [0]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [0]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [0]:
loss = forward(xt2, y_train)

In [0]:
loss.backward()

In [0]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [0]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [0]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [0]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [0]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [0]:
%time loss = model(x_train, y_train)

CPU times: user 137 ms, sys: 4.95 ms, total: 142 ms
Wall time: 70.7 ms


In [0]:
%time model.backward()

CPU times: user 2.84 s, sys: 3.86 s, total: 6.71 s
Wall time: 3.4 s


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [0]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [0]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [0]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [0]:
%time loss = model(x_train, y_train)

CPU times: user 86 ms, sys: 8.25 ms, total: 94.2 ms
Wall time: 46.3 ms


In [0]:
%time model.backward()

CPU times: user 193 ms, sys: 87.6 ms, total: 280 ms
Wall time: 140 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [0]:
%time loss = model(x_train, y_train)

CPU times: user 88.6 ms, sys: 5.04 ms, total: 93.6 ms
Wall time: 46.4 ms


In [0]:
%time model.backward()

CPU times: user 197 ms, sys: 83.9 ms, total: 281 ms
Wall time: 140 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [0]:
#export
from torch import nn

In [0]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [0]:
model = Model(m, nh, 1)

In [0]:
%time loss = model(x_train, y_train)

CPU times: user 85.1 ms, sys: 8.16 ms, total: 93.3 ms
Wall time: 46.3 ms


In [0]:
%time loss.backward()

CPU times: user 135 ms, sys: 78.1 ms, total: 213 ms
Wall time: 71.1 ms


## Export

In [0]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to nb_02.py
